In [23]:
import nltk
from nltk.tokenize import treebank
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 
import math

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import random
from nltk.corpus import opinion_lexicon
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer 
import numpy 
import re

In [2]:
lemmatizer = WordNetLemmatizer() 

In [3]:

def sig(z):
    return (1)/(1 + numpy.exp(-z))

def sig_prime(z):
    return sig(z)*(1-sig(z))

def tanH(z):
    return numpy.tanh(z)

def tanH_prime(z):
    return 1 - ((tanH(z))**2)

In [4]:
dataset = pd.read_csv('deceptive-opinion.csv')
stop_words = set(stopwords.words('english'))

In [5]:
def train_test_split(df,test_size):
    if isinstance(test_size,float):
        test_size = round(test_size * len(df))
        
    if(test_size > len(df)):
        return 0, df

    indices = df.index.tolist()
    test_indices = random.sample(population=indices,k = test_size)

    test_df = df.loc[test_indices]
    train_df = df.drop(test_indices)
    
    return train_df, test_df

train_df,test_df = train_test_split(dataset,0.3)


In [98]:
vocab = {}
for i in range(len(train_df)):
    review = list(train_df['text'])[i]
    words = word_tokenize(review)
    for word in words:
        if word.lower() not in stop_words and len(word) != 1:
            if lemmatizer.lemmatize(word.lower()) not in vocab and re.match('^[a-zA-Z].*$',word.lower()):
                vocab[lemmatizer.lemmatize(word.lower())] = 1
            elif lemmatizer.lemmatize(word.lower()) in vocab and re.match('^[a-zA-Z].*$',word.lower()):
                vocab[lemmatizer.lemmatize(word.lower())] += 1
            

vocab = sorted(vocab.items(), key = 
         lambda kv:(-kv[1], kv[0]))   

finalVocab = []

for i in range(1000):
   
     finalVocab.append(vocab[i][0])

vocab = finalVocab


# old_len_vocab = len(vocab)

# new_len_vocab = 0;

# while(True):
#     old_len_vocab = len(vocab)
   
#     for word in vocab:
#         for synset in wn.synsets(lemmatizer.lemmatize(word.lower())):
#                             for lemma in synset.lemmas():
#                                 if lemma.name() not in vocab:
#                                     vocab.append(lemma.name())



    
#     new_len_vocab = len(vocab);
    
#     if(old_len_vocab == new_len_vocab):
#         break

In [99]:
vocab

['hotel',
 'room',
 'chicago',
 'stay',
 "n't",
 'would',
 'service',
 'great',
 'staff',
 'one',
 'night',
 'stayed',
 'bed',
 'time',
 'like',
 'could',
 'get',
 'u',
 'desk',
 'even',
 'nice',
 'location',
 'day',
 'also',
 'place',
 'clean',
 'back',
 'front',
 'good',
 'bathroom',
 'got',
 'first',
 'experience',
 'view',
 'go',
 'restaurant',
 'never',
 'next',
 'friendly',
 'well',
 'comfortable',
 'floor',
 'made',
 'lobby',
 'really',
 'business',
 'city',
 'two',
 'price',
 'recommend',
 'much',
 'reservation',
 'area',
 'staying',
 'minute',
 'better',
 'check',
 'food',
 'guest',
 'arrived',
 'make',
 'called',
 'trip',
 'say',
 'breakfast',
 'suite',
 'weekend',
 'definitely',
 'went',
 'door',
 'thing',
 'hour',
 'bar',
 'asked',
 'helpful',
 'best',
 'small',
 'found',
 'people',
 'everything',
 'water',
 'booked',
 'way',
 'free',
 'morning',
 'told',
 'beautiful',
 'right',
 'many',
 'problem',
 'another',
 'downtown',
 'took',
 'husband',
 'wife',
 'around',
 'ever',


In [100]:
input_weights = []
hidden_weights = []


for i in range(len(vocab)*2):
    values = []
    for j in range(len(vocab)):
        values.append(numpy.random.uniform(-0.05,0.05))
    input_weights.append(values)

for i in range(2):
    values = []
    for j in range(len(vocab)*2):
         values.append(numpy.random.uniform(-0.05,0.05))
    hidden_weights.append(values)

In [101]:
len(input_weights)

2000

In [102]:
def expectedClass(label):
    if(label == "deceptive"):
        return numpy.array([0.0,1.0])
    else:
        return numpy.array([1.0,0.0])

def inpLayer(input_weights, inputs, m):
    output = []
    for i in range(m):
        summ = sum(numpy.array(input_weights[i])*(inputs))
        
        output.append(summ)
        
    return output

def hiddenLayer(hidden_weights, inputs, m):
    output = []
    for i in range(m):
        summ = sum(numpy.array(hidden_weights[i])*(inputs))
        
        output.append(sig(summ))
    return output
        
def updateWeights(input_weights,hidden_weights,input_output,hidden_output,true_label,learningRate,inputs):
    hiddenOutputError = []
    #print(numpy.array(expectedClass(true_label)) - numpy.array(hidden_output),true_label)
    hidden_weight_error = numpy.array(hidden_output)*(1-numpy.array(hidden_output))*(numpy.array(expectedClass(true_label)) - numpy.array(hidden_output))
    input_weight_error = numpy.array(input_output)*(1-numpy.array(input_output))*((numpy.array(hidden_weights[0])*hidden_weight_error[0]) + (numpy.array(hidden_weights[1])*hidden_weight_error[1]))
    
   # print(input_weight_error,hidden_weight_error)
    for i in range(len(input_weights)):
        input_weights[i] =(input_weights[i]) + numpy.array(inputs)*learningRate*input_weight_error[i] 
        
    for i in range(len(hidden_weights)):
        hidden_weights[i] = (hidden_weights[i]) + numpy.array(input_output)*learningRate*hidden_weight_error[i]

In [103]:
def NN(inputs,input_weights,hidden_weights,hiddenUnits,outputUnits,true_label):
    input_output = inpLayer(input_weights,inputs,hiddenUnits)
    #print(input_output)
    hidden_output = hiddenLayer(hidden_weights,input_output,outputUnits)
   # print(hidden_output)
    updateWeights(input_weights,hidden_weights,input_output,hidden_output,true_label,0.001,inputs)
    
    

In [104]:
def PredictNN(inputs,input_weights,hidden_weights,hiddenUnits,outputUnits,true_label):
    input_output = inpLayer(input_weights,inputs,hiddenUnits)
    #print(input_output)
    hidden_output = hiddenLayer(hidden_weights,input_output,outputUnits)
    return hidden_output

In [105]:
def getWordVector(review):
    vector=[]
    words = [word.lower() for word in word_tokenize(review)]
    for word in vocab:
        if word.lower() in words:
            vector.append(1)
        else:
            vector.append(0)
    return vector

In [106]:
getWordVector(list(train_df['text'])[100]).count(1)

35

In [107]:
for j in range(3):
    
    for i in train_df.index:
        
        inputs = getWordVector(train_df['text'][i])
        
        true_label = train_df['deceptive'][i]
        hiddenUnits = len(input_weights)
        outputUnits = len(hidden_weights)
        NN(inputs,input_weights,hidden_weights,hiddenUnits,outputUnits,true_label)
    
    #print(inputs)
        
    count = 0
    for i in test_df.index:
        inputs = getWordVector(test_df['text'][i])
       # print(inputs)
        true_label = test_df['deceptive'][i]
        hiddenUnits = len(input_weights)
        outputUnits = len(hidden_weights)
        output = PredictNN(inputs,input_weights,hidden_weights,hiddenUnits,outputUnits,true_label)
        if(output.index(max(output)) == 1 and true_label == "deceptive"):
            count += 1
        elif output.index(max(output)) == 0 and true_label == "truthful":
            count += 1
        
    print(j,count/len(test_df))

0 0.6104166666666667
1 0.6520833333333333
2 0.6916666666666667


In [108]:
for j in range(10):
    
    for i in train_df.index:
        
        inputs = getWordVector(train_df['text'][i])
        
        true_label = train_df['deceptive'][i]
        hiddenUnits = len(input_weights)
        outputUnits = len(hidden_weights)
        NN(inputs,input_weights,hidden_weights,hiddenUnits,outputUnits,true_label)
    
    #print(inputs)
        
    count = 0
    truePositive = trueNegative = falsePositive = falseNegative = 0
    for i in test_df.index:
        inputs = getWordVector(test_df['text'][i])
       # print(inputs)
        true_label = test_df['deceptive'][i]
        hiddenUnits = len(input_weights)
        outputUnits = len(hidden_weights)
        output = PredictNN(inputs,input_weights,hidden_weights,hiddenUnits,outputUnits,true_label)
        if(output.index(max(output)) == 1 and true_label == "deceptive"):
            truePositive += 1
        elif output.index(max(output)) == 0 and true_label == "truthful":
            trueNegative += 1
        elif output.index(max(output)) == 0 and true_label == "deceptive":
            falseNegative += 1
        elif output.index(max(output)) == 1 and true_label == "truthful":
            falsePositive += 1
            
    accuracy = (truePositive + trueNegative)/(truePositive + trueNegative + falsePositive + falseNegative)
    precision = truePositive/(truePositive + falsePositive)
    recall =truePositive/(truePositive + falseNegative) 

    
        
    print(accuracy,precision,recall)

0 0.7125
1 0.7395833333333334
2 0.7520833333333333
3 0.7666666666666667
4 0.7791666666666667
5 0.7875
6 0.7895833333333333
7 0.7979166666666667
8 0.7958333333333333
9 0.7979166666666667


In [109]:
truePositive = trueNegative = falsePositive = falseNegative = 0
for i in test_df.index:
    inputs = getWordVector(test_df['text'][i])
   # print(inputs)
    true_label = test_df['deceptive'][i]
    hiddenUnits = len(input_weights)
    outputUnits = len(hidden_weights)
    output = PredictNN(inputs,input_weights,hidden_weights,hiddenUnits,outputUnits,true_label)
    if(output.index(max(output)) == 1 and true_label == "deceptive"):
        truePositive += 1
    elif output.index(max(output)) == 0 and true_label == "truthful":
        trueNegative += 1
    elif output.index(max(output)) == 0 and true_label == "deceptive":
        falseNegative += 1
    elif output.index(max(output)) == 1 and true_label == "truthful":
        falsePositive += 1

accuracy = (truePositive + trueNegative)/(truePositive + trueNegative + falsePositive + falseNegative)
precision = truePositive/(truePositive + falsePositive)
recall =truePositive/(truePositive + falseNegative) 



print(accuracy,precision,recall)

0.7979166666666667 0.7459807073954984 0.928


In [110]:
for j in range(10):
    
    for i in train_df.index:
        
        inputs = getWordVector(train_df['text'][i])
        
        true_label = train_df['deceptive'][i]
        hiddenUnits = len(input_weights)
        outputUnits = len(hidden_weights)
        NN(inputs,input_weights,hidden_weights,hiddenUnits,outputUnits,true_label)
    
    #print(inputs)
        
    count = 0
    truePositive = trueNegative = falsePositive = falseNegative = 0
    for i in test_df.index:
        inputs = getWordVector(test_df['text'][i])
       # print(inputs)
        true_label = test_df['deceptive'][i]
        hiddenUnits = len(input_weights)
        outputUnits = len(hidden_weights)
        output = PredictNN(inputs,input_weights,hidden_weights,hiddenUnits,outputUnits,true_label)
        if(output.index(max(output)) == 1 and true_label == "deceptive"):
            truePositive += 1
        elif output.index(max(output)) == 0 and true_label == "truthful":
            trueNegative += 1
        elif output.index(max(output)) == 0 and true_label == "deceptive":
            falseNegative += 1
        elif output.index(max(output)) == 1 and true_label == "truthful":
            falsePositive += 1
            
    accuracy = (truePositive + trueNegative)/(truePositive + trueNegative + falsePositive + falseNegative)
    precision = truePositive/(truePositive + falsePositive)
    recall =truePositive/(truePositive + falseNegative) 

    
        
    print(accuracy,precision,recall)

0.8104166666666667 0.760655737704918 0.928
0.8125 0.7649006622516556 0.924
0.8125 0.7649006622516556 0.924
0.8145833333333333 0.7674418604651163 0.924
0.8166666666666667 0.77 0.924
0.81875 0.7725752508361204 0.924
0.81875 0.7744107744107744 0.92
0.8166666666666667 0.7736486486486487 0.916
0.81875 0.7762711864406779 0.916
0.81875 0.7762711864406779 0.916


In [114]:
for j in range(10):
    
    for i in train_df.index:
        
        inputs = getWordVector(train_df['text'][i])
        
        true_label = train_df['deceptive'][i]
        hiddenUnits = len(input_weights)
        outputUnits = len(hidden_weights)
        NN(inputs,input_weights,hidden_weights,hiddenUnits,outputUnits,true_label)
    
    #print(inputs)
        
    count = 0
    truePositive = trueNegative = falsePositive = falseNegative = 0
    for i in test_df.index:
        inputs = getWordVector(test_df['text'][i])
       # print(inputs)
        true_label = test_df['deceptive'][i]
        hiddenUnits = len(input_weights)
        outputUnits = len(hidden_weights)
        output = PredictNN(inputs,input_weights,hidden_weights,hiddenUnits,outputUnits,true_label)
        if(output.index(max(output)) == 1 and true_label == "deceptive"):
            truePositive += 1
        elif output.index(max(output)) == 0 and true_label == "truthful":
            trueNegative += 1
        elif output.index(max(output)) == 0 and true_label == "deceptive":
            falseNegative += 1
        elif output.index(max(output)) == 1 and true_label == "truthful":
            falsePositive += 1
            
    accuracy = (truePositive + trueNegative)/(truePositive + trueNegative + falsePositive + falseNegative)
    precision = truePositive/(truePositive + falsePositive)
    recall =truePositive/(truePositive + falseNegative) 

    
        
    print(accuracy,precision,recall)

0.8208333333333333 0.7789115646258503 0.916
0.825 0.7842465753424658 0.916
0.8229166666666666 0.7835051546391752 0.912
0.8229166666666666 0.7835051546391752 0.912
0.825 0.7862068965517242 0.912
0.825 0.7862068965517242 0.912
0.825 0.7862068965517242 0.912
0.8270833333333333 0.7889273356401384 0.912
0.8270833333333333 0.7889273356401384 0.912
0.8270833333333333 0.7889273356401384 0.912


In [115]:
for j in range(30):
    
    for i in train_df.index:
        
        inputs = getWordVector(train_df['text'][i])
        
        true_label = train_df['deceptive'][i]
        hiddenUnits = len(input_weights)
        outputUnits = len(hidden_weights)
        NN(inputs,input_weights,hidden_weights,hiddenUnits,outputUnits,true_label)
    
    #print(inputs)
        
    count = 0
    truePositive = trueNegative = falsePositive = falseNegative = 0
    for i in test_df.index:
        inputs = getWordVector(test_df['text'][i])
       # print(inputs)
        true_label = test_df['deceptive'][i]
        hiddenUnits = len(input_weights)
        outputUnits = len(hidden_weights)
        output = PredictNN(inputs,input_weights,hidden_weights,hiddenUnits,outputUnits,true_label)
        if(output.index(max(output)) == 1 and true_label == "deceptive"):
            truePositive += 1
        elif output.index(max(output)) == 0 and true_label == "truthful":
            trueNegative += 1
        elif output.index(max(output)) == 0 and true_label == "deceptive":
            falseNegative += 1
        elif output.index(max(output)) == 1 and true_label == "truthful":
            falsePositive += 1
            
    accuracy = (truePositive + trueNegative)/(truePositive + trueNegative + falsePositive + falseNegative)
    precision = truePositive/(truePositive + falsePositive)
    recall =truePositive/(truePositive + falseNegative) 

    
        
    print(accuracy,precision,recall)

0.8291666666666667 0.7916666666666666 0.912
0.8270833333333333 0.7929824561403509 0.904
0.825 0.7922535211267606 0.9
0.8270833333333333 0.7950530035335689 0.9
0.8270833333333333 0.7950530035335689 0.9
0.8291666666666667 0.7978723404255319 0.9
0.8333333333333334 0.8057553956834532 0.896
0.8333333333333334 0.8057553956834532 0.896
0.8333333333333334 0.8057553956834532 0.896
0.8354166666666667 0.8086642599277978 0.896
0.8354166666666667 0.8086642599277978 0.896
0.8354166666666667 0.8086642599277978 0.896
0.8354166666666667 0.8086642599277978 0.896
0.8354166666666667 0.8086642599277978 0.896
0.8375 0.8115942028985508 0.896
0.8395833333333333 0.8145454545454546 0.896
0.8395833333333333 0.8145454545454546 0.896
0.8395833333333333 0.8145454545454546 0.896
0.8395833333333333 0.8145454545454546 0.896
0.8395833333333333 0.8145454545454546 0.896
0.8395833333333333 0.8145454545454546 0.896
0.8395833333333333 0.8145454545454546 0.896
0.8416666666666667 0.8175182481751825 0.896
0.8416666666666667 0.

In [ ]:
for j in range(30):
    
    for i in train_df.index:
        
        inputs = getWordVector(train_df['text'][i])
        
        true_label = train_df['deceptive'][i]
        hiddenUnits = len(input_weights)
        outputUnits = len(hidden_weights)
        NN(inputs,input_weights,hidden_weights,hiddenUnits,outputUnits,true_label)
    
    #print(inputs)
        
    count = 0
    truePositive = trueNegative = falsePositive = falseNegative = 0
    for i in test_df.index:
        inputs = getWordVector(test_df['text'][i])
       # print(inputs)
        true_label = test_df['deceptive'][i]
        hiddenUnits = len(input_weights)
        outputUnits = len(hidden_weights)
        output = PredictNN(inputs,input_weights,hidden_weights,hiddenUnits,outputUnits,true_label)
        if(output.index(max(output)) == 1 and true_label == "deceptive"):
            truePositive += 1
        elif output.index(max(output)) == 0 and true_label == "truthful":
            trueNegative += 1
        elif output.index(max(output)) == 0 and true_label == "deceptive":
            falseNegative += 1
        elif output.index(max(output)) == 1 and true_label == "truthful":
            falsePositive += 1
            
    accuracy = (truePositive + trueNegative)/(truePositive + trueNegative + falsePositive + falseNegative)
    precision = truePositive/(truePositive + falsePositive)
    recall =truePositive/(truePositive + falseNegative) 

    
        
    print(accuracy,precision,recall)

0.8479166666666667 0.8241758241758241 0.9
0.8479166666666667 0.8241758241758241 0.9
0.8479166666666667 0.8241758241758241 0.9
0.85 0.8272058823529411 0.9
0.8479166666666667 0.8241758241758241 0.9
0.8479166666666667 0.8241758241758241 0.9


In [ ]:
for j in range(30):
    
    for i in train_df.index:
        
        inputs = getWordVector(train_df['text'][i])
        
        true_label = train_df['deceptive'][i]
        hiddenUnits = len(input_weights)
        outputUnits = len(hidden_weights)
        NN(inputs,input_weights,hidden_weights,hiddenUnits,outputUnits,true_label)
    
    #print(inputs)
        
    count = 0
    truePositive = trueNegative = falsePositive = falseNegative = 0
    for i in test_df.index:
        inputs = getWordVector(test_df['text'][i])
       # print(inputs)
        true_label = test_df['deceptive'][i]
        hiddenUnits = len(input_weights)
        outputUnits = len(hidden_weights)
        output = PredictNN(inputs,input_weights,hidden_weights,hiddenUnits,outputUnits,true_label)
        if(output.index(max(output)) == 1 and true_label == "deceptive"):
            truePositive += 1
        elif output.index(max(output)) == 0 and true_label == "truthful":
            trueNegative += 1
        elif output.index(max(output)) == 0 and true_label == "deceptive":
            falseNegative += 1
        elif output.index(max(output)) == 1 and true_label == "truthful":
            falsePositive += 1
            
    accuracy = (truePositive + trueNegative)/(truePositive + trueNegative + falsePositive + falseNegative)
    precision = truePositive/(truePositive + falsePositive)
    recall =truePositive/(truePositive + falseNegative) 

    
        
    print(accuracy,precision,recall)